In [3]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def preprocess(filepath, imgf, labelf) :
  file = open(filepath, 'r')
  for line in file.readlines():
    img, label = line.split()
    imgf.append('./data/'+img); labelf.append(label)
  file.close()

In [3]:
def cv2hog(imgl):
  histl = []
  winSize = (64,64)
  blockSize = (16,16)
  blockStride = (8,8)
  cellSize = (8,8)
  nbins = 9
  derivAperture = 1
  winSigma = 4.
  histogramNormType = 0
  L2HysThreshold = 2.0000000000000001e-01
  gammaCorrection = 0
  nlevels = 64
  for ig in imgl:
    image = cv2.imread(ig,0)
    # plt.imshow(image)
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    hist = hog.compute(image, winStride=(8,8), padding=(8,8), locations=((10,20),))
    histl.append(hist)
  return(histl)

In [4]:
test_y = []
test_imgs = []
train_y = []
train_imgs = []
val_y = []
val_imgs = []
preprocess('./data/test.txt',test_imgs,test_y)
preprocess('./data/train.txt',train_imgs,train_y)
preprocess('./data/val.txt',val_imgs,val_y)

In [5]:
test_imgs

['./data/images/n02111277/n02111277_9420.JPEG',
 './data/images/n02111277/n02111277_9422.JPEG',
 './data/images/n02111277/n02111277_9484.JPEG',
 './data/images/n02111277/n02111277_951.JPEG',
 './data/images/n02111277/n02111277_9518.JPEG',
 './data/images/n02111277/n02111277_9562.JPEG',
 './data/images/n02111277/n02111277_9619.JPEG',
 './data/images/n02111277/n02111277_9622.JPEG',
 './data/images/n02111277/n02111277_9641.JPEG',
 './data/images/n02111500/n02111500_9386.JPEG',
 './data/images/n02111500/n02111500_9398.JPEG',
 './data/images/n02111500/n02111500_9403.JPEG',
 './data/images/n02111500/n02111500_9405.JPEG',
 './data/images/n02111500/n02111500_9409.JPEG',
 './data/images/n02111500/n02111500_9410.JPEG',
 './data/images/n02111500/n02111500_9418.JPEG',
 './data/images/n02111500/n02111500_9420.JPEG',
 './data/images/n02111500/n02111500_9423.JPEG',
 './data/images/n02111889/n02111889_9654.JPEG',
 './data/images/n02111889/n02111889_97.JPEG',
 './data/images/n02111889/n02111889_9705.JP

In [6]:
train_hog = np.array(cv2hog(train_imgs))
val_hog = np.array(cv2hog(val_imgs))
test_hog = np.array(cv2hog(test_imgs))

In [5]:
print(np.shape(train_hog))
print(np.shape(train_y))
print(np.shape(val_hog))
print(np.shape(val_y))
print(np.shape(test_hog))
print(np.shape(test_y))

(63325, 1764)
(63325,)
(450, 1764)
(450,)
(450, 1764)
(450,)


In [14]:
train_hog = train_hog.reshape(63325, 1764)
val_hog = val_hog.reshape(450, 1764)
test_hog = test_hog.reshape(450, 1764)

In [6]:
print(type(train_hog))
print(type(train_y))
print(type(test_hog))
print(type(test_y))
print(type(val_hog))
print(type(val_y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [9]:
train_y = np.array(train_y)
val_y = np.array(val_y)
test_y = np.array(test_y)

In [17]:
np.save('train_hog.npy', train_hog)
np.save('train_y.npy', train_y)
np.save('val_hog.npy', val_hog)
np.save('val_y.npy', val_y)
np.save('test_hog.npy', test_hog)
np.save('test_y.npy', test_y)

In [4]:
test_hog = np.load('./test_hog.npy')
test_y = np.load('./test_y.npy')
train_hog = np.load('./train_hog.npy')
train_y = np.load('./train_y.npy')
val_hog = np.load('./val_hog.npy')
val_y = np.load('./val_y.npy')

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
print(train_hog[1])

[0.10831746 0.12597914 0.13383391 ... 0.25205755 0.25205755 0.25205755]


In [34]:
print(63325*0.0025)
print(63325*0.001)

158.3125
63.325


In [61]:
rf_clf = RandomForestClassifier(n_estimators=42, criterion='entropy', min_samples_split=150, min_samples_leaf=30)
rf_clf.fit(train_hog, train_y)

RandomForestClassifier(criterion='entropy', min_samples_leaf=30,
                       min_samples_split=150, n_estimators=42)

In [65]:
from sklearn.model_selection import GridSearchCV

In [68]:
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
# Create the parameter grid based on the results of random search 
param_grid = {
    'criterion': ['entropy'],
    'bootstrap': [True],
    #'max_depth': [30, 50, 70],
    'min_samples_leaf': [10, 30, 50],
    'min_samples_split': [60, 100, 200],
    'n_estimators': [42, 100, 200]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = 6, verbose = 4)

In [69]:
grid_search.fit(train_hog, train_y)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed: 144.6min
[Parallel(n_jobs=6)]: Done  81 out of  81 | elapsed: 901.7min finished


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=6,
             param_grid={'bootstrap': [True], 'criterion': ['entropy'],
                         'min_samples_leaf': [10, 30, 50],
                         'min_samples_split': [60, 100, 200],
                         'n_estimators': [42, 100, 200]},
             verbose=4)

In [10]:
import joblib
# https://stackoverflow.com/questions/51424312/how-to-save-gridsearchcv-object
#save your model or results
# joblib.dump(grid_search, 'grid_search1.pkl')

#load your model for further usage
grid_search = joblib.load("grid_search1.pkl")

In [11]:
#https://stackoverflow.com/questions/58835592/result-of-gridsearchcv-as-table
df = pd.DataFrame(grid_search.cv_results_)
print(df)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0     1516.603474      7.620692         1.025770        0.012320   
1     3620.778766     16.162968         2.119717        0.020914   
2     7217.565835     46.357504         4.226512        0.097903   
3     1470.730996     15.318884         0.915171        0.015874   
4     3532.512791     29.523535         2.198563        0.093412   
5     7035.881874     25.881440         4.030731        0.069327   
6     1398.279723      2.233094         0.894465        0.066068   
7     3353.153974      4.986882         2.059710        0.117003   
8     6637.047816     11.974087         3.737543        0.017128   
9     1440.925207      4.022443         0.903127        0.009549   
10    3424.138032      7.807457         2.066025        0.033078   
11    6823.665420     16.377056         4.056187        0.058545   
12    1419.499871      5.084598         0.895637        0.009237   
13    3371.248297      6.770593         1.990074

In [12]:
print(np.mean(grid_search.predict(val_hog)==val_y))
print(np.mean(grid_search.predict(train_hog)==train_y))
print(np.mean(grid_search.predict(test_hog)==test_y))

0.05333333333333334
0.7503671535728386
0.057777777777777775


In [62]:
np.mean(rf_clf.predict(val_hog)==val_y)

0.051111111111111114

In [63]:
np.mean(rf_clf.predict(train_hog)==train_y)

0.4475167785234899

In [64]:
np.mean(rf_clf.predict(test_hog)==test_y)

0.03111111111111111

In [13]:
%matplotlib inline

In [90]:
from sklearn.svm import SVC

In [92]:
# https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
svm = SVC(kernel='rbf',gamma='scale', max_iter=-1, probability=True)

In [24]:
svm.fit(train_hog, train_y)

NameError: name 'svm' is not defined

In [23]:
print(np.mean(svm.predict(val_hog)==val_y))
print(np.mean(svm.predict(train_hog)==train_y))
print(np.mean(svm.predict(test_hog)==test_y))

NameError: name 'svm' is not defined

In [21]:
from sklearn import __version__
__version__

'0.23.2'

In [22]:
sklearn.__version__

'0.23.2'

In [18]:
from sklearn.metrics import top_k_accuracy_score

ImportError: cannot import name 'top_k_accuracy_score'

In [75]:
from sklearn import linear_model

# 大量資料用SVM沒有效率
# 用PCA是不是也是
# 所以通常會先用Lasso降維
# random state 是什麼

In [77]:
las_clf = linear_model.Lasso(alpha=1, fit_intercept=True, normalize=False)
las_clf.fit(train_hog, train_y)


Lasso(alpha=1)

In [79]:
print(las_clf.coef_)
print(las_clf.intercept_)

[-0.  0.  0. ...  0.  0. -0.]
24.674772


In [89]:
np.where(las_clf.coef_!=0)

(array([], dtype=int64),)

In [80]:
print(np.mean(las_clf.predict(val_hog)==val_y))
print(np.mean(las_clf.predict(train_hog)==train_y))
print(np.mean(las_clf.predict(test_hog)==test_y))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


0.0
0.0
0.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until
